# Deep Feedforward Networks

## Introduction and Basic Terminology

**Deep feedforward networks**, also called **feedforward networks** or **multilayer perceptrons** (MLPs), are the original deep learning model. The goal of a feedforward network is to approximate a function $f^*$, defining a mapping $\textbf(y)=f(x;\theta)$ that best learns the parameters $\theta$ to approximate $f^*$

These models are called feedforward because the information flows through the function being evaluated from $x$, through the function being evaluated, and to its output $y$. While other neural network models have recurrent connections, in a feedforward network information only flows forward.

These models are called networks because they typically comprise many different functions. These are often represented by computational graphs to show the direction of information flow. Given $f^{1}$, $f^{2}$, and $f^{3}$, for example, a simple feedforward network can be defined as $ f^{3}(f^{2}(f^{1}(X)))$, where the output of $f^{1}$ feeds into $f^{2}$, which is then fed into $f^{3}$ to produce output $y$. The different functions, in neural network terms, will be referred to as **layers**.

In this network, the initial layer, which receives the raw input data, is called the **input layer**. The final layer tha produces output $y$ is called the **output layer**. Every other layer, which both receives its input and feeds its output to another layer in the network, is referred to as a **hidden layer** because we do not directly observe its output.

Last, each layer is represented by a vector of weights that the function applies to data that passes through that layer. The size of these vectors is called the layer's **width**. These weight vector values are analagous to the **neurons** in a biological neural network, and will often be referred to as such. 

## Gradient-Based Learning

Training a neural network is similar to training a standard machine learning model with gradient descent. The biggest difference between neural networks and linear models is that the nonlinearity of a neural network causes most loss functions to become nonconvex, meaning that they must be trained iteratively with gradient-based optimizers rather than linear equation solvers with global convergence guarantees. 

### Cost Functions

In most cases, our parametric model defines a distribution $ p(\textbf y|\textbf x;\theta) $ and we use the principle of maximum likelihood. This means we use the cross-entropy between the training data and the model's predictions as the cost function. Refer back to the notes on information theory and you will see that a lower cross entropy means that the two distributions are more similar, and therefore the model is doing a better job of approximating $f^*$ as the cross-entropy decreases. 

Sometimes, we take a simpler approach, predicting some statistic of $\textbf y$ conditioned on $x$ instead of predicting a complete probability distribution over $\textbf y$. 

The total cost function of a neural network will combine one of these primary cost functions with a regularization term. Model regularization, however, will be discussed in a later notebook.

#### Learning Conditional Distributions with Maximum Likelihood

Most modern neural networks are trained using maximum likelihood. This means that the cost function is simply the negative log-likelihood, equivalently described as the cross-entropy between the training data and the model distribution. This cost function is defined as:

$$ J(\theta) = -E_{x,y \tilde{} \hat{P}_{data}}[ \text{log}p_{\text{model}}(\textbf y | \textbf x) ]. $$

The specific form of the cost function changes from model to model depending on the specific form of $ p_{\text{model}}. $ Some of the terms in the above equation will typically not depend on the model parameters and can be discarded. 

The cost function should be large and predictable enough that it can be used to guide learning. Functions that saturate are bad for this reason because their gradients become small. The negative log likelihood is therefore helpful, because its log function undoes the exp functions of some output units. 

#### Learning Conditional Statistics

We sometimes want to learn a single conditional statistic $\textbf y$ instead of a full probability distribution $ p(\textbf y | \textbf x). $ The mean squared error is one example of such a statistic:


$$ f^* = \text{arg min} E_{x,y \tilde{} \hat{P}_{data}}||\textbf y - f(\textbf x)||^2, $$

which yields

$$ f^* = E_{x,y \tilde{} \hat{P}_{data}(\textbf y | \textbf x}[\textbf y].  $$

This shows that, if we are able to train over infinitely many sample from the data generating mechanism, minimizing the MSE will give us the mean of $\textbf y$ for each value of $\textbf x.$ Unfortunately, however, the mean and median squared error cost functions typically perform poorly in neural network models when used with gradient-based optimzation. This is because some output units tend to saturate and produce small gradients when combined with these cost functions. For this reason, the cross-entropy is most common.

### Output Units

The choice of cost functino is tightly coupled with the choice of output unit. We will typically use the cross-entropy between the data and model distributions as our cost function. The output unit, therefore, will determine the form that this cross entropy takes. 

#### Linear Units for Gaussian Output Distributions

A simple output unit is a **linear unit** that is simply based on an affine transoformation. 

Given features $\textbf h$, a layer of linear output units produces a vector $\mathbf{\hat{\text{y}}} = \textbf W^{T}\textbf h + \textbf b.$ Linear output layers are used to produce the mean of a conditional Gaussian distribution:

$$ p(\textbf y | \textbf x) = N(\textbf y; \mathbf{\hat{\text{y}}}, \textbf I). $$

Maximizing the log likelihood of this is then equivalent to minimizing MSE.

#### Sigmoid Units for Bernoulli Output Distributions

The maximum likelihood approach to estimating a binary variable $y$ is to define a Bernoulli distribution over $y$ conditioned on $x$. The neural network, in this case, must only predict $p(y=1|\textbf x)$ with the constraint that y is between 0 and 1. 

A linear output unit will not perform well here, since forcing its output to interval [0,1] in some places will cause the gradient to saturate to zero. This is why we use a squashing function such as the logistic sigmoid. 

A sigmoid output is defined by:

$$ \hat{y} = \sigma \left(  \textbf{w}^{T}\textbf h + b \right), $$

where \sigma is the logistic sigmoid function:

$$ \sigma(x) = \frac{1}{1+\text{exp}(-x)}. $$

We can construct an unnormalized probability distribution $\tilde{P}(y)$ by exponentiating the log probabilities, so long as we assume that $P$ is linear in $y$ and $z$, where $z = \textbf{w}^{T}\textbf h + b.$ We can then divide by the appropriate constant so that the values sum to 1, which yields a Bernoulli distribution controlled by a sigmoidal transformation of $z$. $z$ here is called a **logit**.

$$ \text{log}\tilde{P}(y) = yz, $$

$$ \tilde{P}(y) = \text{exp}(yz), $$

$$ P(y) = \frac{\text{exp}(yz)}{\sum_{y^{'}=0}^1 \text{exp}(y^{'}z)},$$

$$ P(y) = \sigma((2y - 1)z). $$

This approach to predicting probabilities in log space is natural to use with maximum likelihood learning because the log in $J(\theta)=-\text{log}P(y|\textbf x)$ undoes the exp of the sigmoid. This prevents saturation and allows gradient-based training to work.

The loss function of maximum likelihood learning of a Bernoulli parameterized by a sigmoid is:

$$ J(\theta)=-\text{log}P(y|\textbf x), $$
$$ J(\theta)=-\text{log}\sigma((2y - 1)z), $$
$$ \zeta((2y - 1)z), $$

where $\zeta$ is the softplus function. This has the helpful property that its gradient only saturates as x becomes very small, in which case the model has already found the right answer. With a different loss function, such as MSE, the exp would remain in the loss function, causing it to saturate whenever $\sigma(z)$ does.

#### Softmax Units for Multinoulli Output Distributions

When we wish to represent a probability distribution over $n$ possible discrete values, we typically use the softmax function. This is a generalization of the sigmoid function to work with multiple classes. 

With multiple classes, we now want to predict $ \hat{y}_{i} = P(y = i | \textbf x) $ for each class $(1, . . ., n).$ Each value must be between 0 and 1, and the entire vector $\mathbf{\hat{y}}$ or $n$ elements sums to 1. The same approach that worked for the Bernoulli distribution generalizes to the multinoulli distribution. First, a linear layer predicts unnormalized log probabilities:

$$ z = \textbf{W}^{T}\textbf{h} + \textbf{b}, $$

where $z_{i} = \text{log}\tilde{P}(y=i|\textbf x).$ The softmax function can then exponentiate and normalize $z$ to obtain the desired $\mathbf{\hat{y}}.$ Formally, the softmax function is defined as:

$$ \text{softmax}(z)_{i} = \frac{\text{exp}(z_{i})}{\sum_{j} \text{exp}(z_{j})}. $$

As with the sigmoid function, this trains well when using maximum likelihood estimation, with the log undoing the exp in the softmax:

$$ \text{log softmax}(z)_{i} = z_{i} - \text{log}\sum_{j} \text{exp}(z_{j}). $$

## Hidden Units
The existing body of theory on neural network hidden units is largely experimental, with relatively few guiding principles. Rectified Linear Units (ReLUs) are generally the default in modern deep learning literature, but there are various others that can also work well. Some of these funcitons are not actually differentiable when equal to 0, but this does not tend to pose a problem in practice, because training does not typically reach this point. 

### Rectified Linear Units and Their Generalizations (ReLU)

The **ReLU** activation function is defined as:

$$ g(z) = \text{max}\{ 0, z \}. $$

Thes units are easy to optimze because they are similar to linear units, but are superior to linear units because of the nonlinearity imposed by setting them equal to zero across half their domain. This makes the derivative of this unit large whenever the unit is active. 

Rectified linear units are typically used on top of an affine transformation:

$$ \textbf h = g(\textbf{W}^{T} \textbf{x} + \textbf{b}). $$

One drawback to the ReLU is that it cannot learn by griadient-based methods where its activation is zero. This has motivated the use of various generalizations of the ReLU that smooth the function so that its derivative is always nonzero. For further reading on these, see the **leaky ReLU**, **parametric ReLU** (PReLU), **absolute value rectification**, and **maxout units**. 

### Logistic Sigmoid and Hyperbolic Tangent

Prior to the introduction of the ReLU activation function, most networks used either sigmoid activation:

$$ g(x) = \sigma(z) $$

or hyperbolic tangent activation:

$$ g(z) = \text{tanh}(z). $$

These functions are closely related because $\text{tanh}(z) = 2\sigma(2z)-1.$ 

Unlike piecewise activation units such as ReLU, sigmoidal units saturate across most of their domain. This makes gradient-based learning difficult. 


### Other Hidden Units

A few other hidden units sometimes used in practice are:

* radial basis functions (RBFs)
* softplus
* hard tanh

## Architecture Design

A key design consideration for a neural network is its architecture. A feedforward network's architecture typically refers to its number of layers of hidden units, its number of hidden units per layer, and how they units should be connected with each other. 

Most networks are constructed of chained layers, with each layer receiving a previous layer's output as its input. The function of the first layer is given by

$$ h^{(1)} = g^{(1)}\left( W^{(1)T}x + b^{(1)} \right), $$

The second's by

$$ h^{(2)} = g^{(2)}\left( W^{(2)T}h^{(1)} + b^{(2)} \right), $$

and so on. 

In this chained structure, our primary foci are choosing the best number of layers and number of hidden units per layer. 

### Universal Approximation Properties and Depth

A linear funciton, mapping features to outputs via matrix multiplication, can by definition only represent only linear functions. We often, however, want to learn a function that is nonlinear. 

**Universal approximation theorem** (Hornik, Stinchcombe and White '89) states that a feedforward network with a linear output layer, at least one hidden layer, and any "squashing" activation function, can approximate any Borel measurable function from one finite-dimensional space to another with any desired nonzero amount of error, provided that the network is given enough hidden units. This theorem was later generalized to non-sigmoidal activation functions such as the ReLU. 

A few caveats to this:

* While the MLP may be able to represent the function, it won't necessarily learn the weights of the true underlying function 
* The learning algorithm may represent the wrong function as a result of overfitting
* While this theorem says we can achieve any degree of accuracy we desire by changing the number of hidden units, it does not say how many units we will actually need

Unfortunately, the number of hidden units needed may in some cases be massive. Using deeper models, however, can reduce the number of units needed to represent a function, at the the expense of training time.  





### Other Architectural Considerations

In practice, not all networks are chained this way. Recurrent networks have a structure that feeds back into itself recursively, for example. Other networks feature skip connections that go from layer $i$ to layer $i+2$ or higher, allowing the gradient to better flow from output to input. In general, however, this chained structure is standard. 

Another consideration is how to connect the units between layers. The common case is to perform a linear transformation via a matrix $\textbf W,$ where each neuron in one layer connects to each neuron in the next. Some networks, however, use fewer connections to save on computation cost. Convolutional networks are an example of a network applying a specialized, sparser system of connections between layers. 

## Backpropogation and Other Differentiation Algorithms

The flow of information from a network's input layer, through its transformations and hidden layers, toward an output, is called **forward propogation.** To train a model, we forward-propogate to retrieve an output and scalar cost $J(\theta)$. Given this cost, we then must **backpropogate** through the network to compute the gradient and update the model's weights in order to improve its cost. 


### Chain Rule of Calculus

Backpropogation is an algorithm that computes the chain rule in an efficient manner. Where $y = g(x)$ and $z = f(g(x))$, the chain rule states that:

$$ \frac{dz}{dx} = \frac{dz}{dy} \frac{dy}{dx}. $$

This can be generalized beyond the scalar case. In vector notation, this can be written as:

$$ \triangledown_{x}z = \left( \frac{dy}{dz} \right)^T \triangledown_{y}z, $$

where the term in parentheses is the n x m Jacobian matrix of g. Backprop, then, consists of performing a Jacobian-gradient product for each operation in the computational graph. 

### Backpropogation Computation in Fully Connected MLPs

In [ ]:
#forward propogation pseudocode
h_0 = x
for k in range(network_depth):
    a_k = b_k + W_k*h_kminus1
    h_k = f(a_k)
yhat = h_k
J = loss(yhat, y) + reg_lambda*gamma(theta)

In [ ]:
#backpropogation pseudocode
g = change_in_loss_function
for k in range(network_depth):
    g = g*derivative_of_activation(a_k)
    delta_J = g + reg_lambda*change_in_gamma(theta)
    g = W*g